In [1]:
import os
import torchaudio
import torch
from EgoExoEMS.EgoExoEMS import EgoExoEMSDataset, collate_fn, transform
import numpy as np
import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")



In [2]:
root = "/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng9/cardiac_arrest/1/GoPro/GX010346_encoded_trimmed.mp4"  # Folder in which all videos lie in a specific structure
annotation_file = "../../Annotations/main_annotation_classification.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)

# train_annotation_file = "../../Annotations/splits/keysteps/train_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)
# val_annotation_file = "../../Annotations/splits/keysteps/val_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)
# test_annotation_file = "../../Annotations/splits/keysteps/test_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)


train_dataset = EgoExoEMSDataset(annotation_file=annotation_file,
                                data_base_path='',
                                fps=29.97, frames_per_clip=None, transform=transform, data_types=[ 'resnet','audio','smartwatch','depth_sensor'])

# Access a sample
print(len(train_dataset))






[Warning] Skipping trial 1 for subject wa1 due to missing data
1765


In [3]:
flow_path = '/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng3/cardiac_arrest/9/i3d_flow/GX010374_encoded_trimmed_flow.npy'
gopro_path = '/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ng3/cardiac_arrest/9/GoPro/GX010374_encoded_trimmed.mp4'

# load the flow
flow = np.load(flow_path)
print(flow.shape)

import math

print((2724-2626))
print(math.ceil((2724-2626)/30))

(3371, 1024)
98
4


In [4]:
# import torchvision
# from torchvision.io import VideoReader
# import os
# import itertools

# start_t = 71.30542
# end_t = 72.993
# video_path = root
# # video_path = "/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/Sep_2024/Raw/24-09-2024/Bhavik/cardiac_arrest/4/GoPro/GX010399.MP4"  # Folder in which all videos lie in a specific structure
# video_reader = VideoReader(video_path, "video")
# frames = []


# for frame in itertools.takewhile(lambda x: x['pts'] <= end_t, video_reader.seek(start_t)):
#     if(frame['pts'] < start_t):
#         continue
#     print(frame['pts'])
#     img_tensor = transform(frame['data'])
#     frames.append(img_tensor)

# frames = torch.stack(frames)
# print("Seeking from ", start_t, " to ", end_t, "for video ", video_path)
# print("Frames shape: ", frames.shape)

In [5]:
# create a data loader
# batch size is 1 for simplicity and to ensure only a full clip related to a key step is given without collating.
# if batch size is greater than 1, collate_fn will be called to collate the data.
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)
print(len(data_loader))

353


In [6]:
# Iterate over the data loader and print the shape of the batch
for batch in data_loader:
    print(batch['frames'].shape, batch['audio'].shape,batch['resnet'].shape,batch['resnet_exo'].shape, batch['flow'].shape, batch['rgb'].shape, batch['smartwatch'].shape, batch['depth_sensor'].shape , batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'],batch['start_t'], batch['end_t'],  batch['subject_id'], batch['trial_id'])
    print("*"*4 + "="*50 + "*"*4)
    # break
    
    # audio_tensor = batch['audio'][0]
    # #transpose
    # audio_tensor = audio_tensor.transpose(0,1)
    # torchaudio.save("./visualizations/audio.wav", audio_tensor,48000)
    # break   

torch.Size([0]) torch.Size([5, 3261440, 2]) torch.Size([5, 2036, 2048]) torch.Size([5, 0]) torch.Size([5, 0]) torch.Size([5, 0]) torch.Size([5, 2036, 3]) torch.Size([5, 2036, 1]) ['approach_patient', 'check_responsiveness', 'check_pulse', 'check_breathing', 'chest_compressions'] tensor([0, 1, 2, 3, 4]) tensor([  0, 198, 342, 401, 472]) tensor([ 198,  342,  472,  472, 2508]) tensor([ 0.0000,  6.6090, 11.4190, 13.3950, 15.7730]) tensor([ 6.6086, 11.4189, 15.7731, 15.7731, 83.7072]) ['ms1', 'ms1', 'ms1', 'ms1', 'ms1'] ['0', '0', '0', '0', '0']
****==================================================****
torch.Size([0]) torch.Size([5, 260096, 2]) torch.Size([5, 162, 2048]) torch.Size([5, 0]) torch.Size([5, 0]) torch.Size([5, 0]) torch.Size([5, 162, 3]) torch.Size([5, 162, 1]) ['request_assistance', 'request_aed', 'no_action', 'turn_on_aed', 'no_action'] tensor([ 6,  5, 15,  7, 15]) tensor([ 584,  674,  752,  914, 1006]) tensor([ 674,  752,  914, 1006, 1015]) tensor([19.5090, 22.5090, 25.1139

## Torchaudio feature extraction

In [ ]:
import torch
import torchaudio
import torchaudio.transforms as transforms

# Example batch of audio clips (batch, samples, channels)
audio_clips = batch['audio']  # Assume shape [batch, samples, channels]

# Parameters for feature extraction
sample_rate = 48000  # Adjust based on your dataset
n_mels = 64  # Number of Mel filter banks
hop_length = 512
n_fft = 1024

# MelSpectrogram transform
mel_spectrogram = transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_mels=n_mels,
    hop_length=hop_length,
    n_fft=n_fft
)

# Process audio for one channel (if you want stereo/multi-channel, handle each channel accordingly)
batch_size, num_samples, num_channels = audio_clips.shape

# If using mono audio (process first channel as an example)
mel_features = mel_spectrogram(audio_clips[:, :, 0])  # Shape: [batch, n_mels, time]

# To get the desired shape [batch, samples, feature_dim], we treat the time axis as the "samples"
# Here, n_mels will be the feature dimension and time will be the new samples axis
mel_features = mel_features.permute(0, 2, 1)  # Shape: [batch, time(samples), n_mels(feature_dim)]

print(f"Mel-spectrogram features shape: {mel_features.shape}")


### Dual channel mel-spectrogram

In [ ]:
# Example batch of audio clips (batch, samples, channels)
audio_clips = batch['audio']  # Assume shape [batch, samples, channels]

# Parameters for feature extraction
sample_rate = 48000  # Adjust based on your dataset
n_mels = 64  # Number of Mel filter banks
hop_length = 512
n_fft = 1024

# MelSpectrogram transform
mel_spectrogram = transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_mels=n_mels,
    hop_length=hop_length,
    n_fft=n_fft
)

# Extract Mel-spectrogram for both channels
mel_spec_left = mel_spectrogram(audio_clips[:, :, 0])  # Left channel
mel_spec_right = mel_spectrogram(audio_clips[:, :, 1])  # Right channel

mel_spec_combined = torch.cat((mel_spec_left, mel_spec_right), dim=-1)  # Shape: [n_mels, time*2]
print(f"Mel-spectrogram combined shape: {mel_spec_combined.shape}")


# Convert to log scale
mel_spec_left_db = transforms.AmplitudeToDB()(mel_spec_left)
mel_spec_right_db = transforms.AmplitudeToDB()(mel_spec_right)


# Concatenate the spectrograms side by side (along the time axis)
mel_spec_combined_db = torch.cat((mel_spec_left_db, mel_spec_right_db), dim=-1)  # Shape: [n_mels, time*2]
print(f"Mel-spectrogram combined shape: {mel_spec_combined_db.shape}")


In [ ]:
import torch
import torchaudio
import torchaudio.transforms as transforms
import matplotlib.pyplot as plt

# Example batch of audio clips (batch, samples, channels)
audio_clips = batch['audio']  # Assume shape [batch, samples, channels]

# Parameters for feature extraction
sample_rate = 16000  # Adjust based on your dataset
n_mels = 64  # Number of Mel filter banks
hop_length = 512
n_fft = 1024

# MelSpectrogram transform
mel_spectrogram = transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_mels=n_mels,
    hop_length=hop_length,
    n_fft=n_fft
)

# Extract Mel-spectrogram for a single channel (first sample, first channel as an example)
mel_spec = mel_spectrogram(audio_clips[0, :, 0])  # Shape: [n_mels, time]

# Convert to log scale for better visualization
mel_spec_db = transforms.AmplitudeToDB()(mel_spec)

# Plotting the Mel-spectrogram
plt.figure(figsize=(10, 6))
plt.imshow(mel_spec_db.numpy(), cmap='viridis', aspect='auto', origin='lower')
plt.colorbar(format="%+2.0f dB")
plt.title("Mel-Spectrogram (Log-Scale)")
plt.xlabel("Time (frames)")
plt.ylabel("Mel Frequency Bins")
plt.tight_layout()
plt.show()


In [ ]:
# convert frames tensor to video
frames = batch['frames'][0]
print(frames.shape) # (num_frames, 3, 224, 224)

import torch
import cv2
import numpy as np

# Example tensor (num_frames, 3, 224, 224)
frames = batch['frames'][0]

# Convert from (num_frames, 3, 224, 224) to (num_frames, 224, 224, 3)
frames = frames.permute(0, 2, 3, 1).cpu().numpy()  # (num_frames, 224, 224, 3)

# Normalize pixel values from [0, 1] or [-1, 1] if necessary
# frames = (frames * 255).astype(np.uint8)

# convert bgr to rgb
# Define the codec and create a VideoWriter object
output_file = './visualizations/video.mp4'
fps = 30  # Frames per second
height, width = frames.shape[1:3]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4

video_writer = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

# Write each frame to the video
for frame in frames:
    video_writer.write(frame)

# Release the video writer
video_writer.release()

print(f"Video saved as {output_file}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_rgb_flow_imu_depth(rgb_frames, flow_frames, rgb_feature, imu_data, depth_data):
    # Assume rgb_frames is of shape (frames, height, width, 3) and flow_frames is (frames, 1024)
    # imu_data is of shape (frames, 3) and depth_data is (frames, 1)
    print(rgb_frames.shape, flow_frames.shape, rgb_feature.shape, imu_data.shape, depth_data.shape)
    num_frames = rgb_frames.shape[0]

    fig, axes = plt.subplots( 5, num_frames, figsize=(num_frames * 5, 25))

    for i in range(num_frames):
        rgb_image = rgb_frames[i].permute(1, 2, 0).cpu().numpy()
        
        # Plot RGB frame
        axes[0, i].imshow(rgb_image)
        axes[0, i].axis('off')
        axes[0, i].set_title(f"RGB Frame {i+1}")
        
        # Plot flow data as a heatmap for the corresponding frame
        axes[1, i].imshow(flow_frames[i].reshape(32, 32), cmap='viridis', aspect='auto')  # assuming 1024 is reshaped to 32x32
        axes[1, i].axis('off')
        axes[1, i].set_title(f"Flow Feature {i+1}")
        
        # Plot the RGB feature as a heatmap
        rgb_feature_image = rgb_feature[i].cpu().numpy().reshape(32, 32)  # Reshape to 32x32
        axes[2, i].imshow(rgb_feature_image, cmap='plasma', aspect='auto')
        axes[2, i].axis('off')
        axes[2, i].set_title(f"RGB Feature {i+1}")
        
        # Plot smartwatch IMU data (3-axis)
        # Plot smartwatch IMU data (3-axis: x, y, z)
        imu_time_series = imu_data[i].cpu().numpy()  # Shape (3,)
        axes[3, i].plot([0, 1, 2], imu_time_series, marker='o', label=['X', 'Y', 'Z'])
        axes[3, i].set_xticks([0, 1, 2])
        axes[3, i].set_xticklabels(['X', 'Y', 'Z'])
        axes[3, i].set_title(f"IMU Data {i+1}")
        
        # Plot depth sensor data
        depth_value = depth_data[i].cpu().numpy()
        axes[4, i].bar(0, depth_value, width=0.5)
        axes[4, i].set_ylim(0, np.max(depth_data.cpu().numpy()))  # Adjust y-axis based on max depth value
        axes[4, i].set_title(f"Depth Sensor {i+1}")
        axes[4, i].set_xticks([])  # No x-ticks since it's a single bar

        # Save each figure
        plt.savefig(f"./visualizations/frame_{i}.png")

    plt.tight_layout()
    plt.show()

# Example usage with your batch data
# batch = next(iter(data_loader))
print(batch['frames'].shape, batch['audio'].shape, batch['flow'].shape, batch['rgb'].shape, batch['smartwatch_imu'].shape, batch['depth_sensor'].shape , batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'],batch['start_t'], batch['end_t'],  batch['subject_id'], batch['trial_id'])
# torch.Size([1, 82, 3, 224, 224]) torch.Size([1, 133120, 2]) torch.Size([1, 1, 1024]) torch.Size([1, 1, 1024]) torch.Size([1, 82, 3]) torch.Size([1, 82, 1]) ['place_bvm'] tensor([13]) tensor([2416]) tensor([2498]) tensor([80.6273]) tensor([83.3773]) ['ng8'] ['7']

rgb_frames= batch['frames'][0]
flow_frames = batch['flow'][0]
rgb_feature = batch['rgb'][0]
imu_data = batch['smartwatch_imu'][0]
depth_data = batch['depth_sensor'][0]

plot_index = min(rgb_frames.shape[0], flow_frames.shape[0], rgb_feature.shape[0], imu_data.shape[0], depth_data.shape[0])

if(plot_index > 1):
    plot_index = 15
    plot_rgb_flow_imu_depth(rgb_frames[0:plot_index], flow_frames[0:plot_index], rgb_feature[0:plot_index],imu_data[0:plot_index], depth_data[0:plot_index])


In [ ]:
! ffmpeg

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation

def animate_rgb_flow_imu_depth(rgb_frames, flow_frames, additional_rgb, imu_data, depth_data):
    num_frames = rgb_frames.shape[0]
    
    fig, axes = plt.subplots(5, 1, figsize=(6, 15))  # 5 rows for RGB, Flow, Additional RGB, IMU, and Depth

    # Initialize the images that will be updated
    rgb_im = axes[0].imshow(np.zeros((rgb_frames.shape[2], rgb_frames.shape[3], 3)))
    flow_im = axes[1].imshow(np.zeros((32, 32)), cmap='viridis')
    add_rgb_im = axes[2].imshow(np.zeros((32, 32)), cmap='plasma')
    
    # For IMU and Depth, we initialize placeholders
    imu_plot_x, = axes[3].plot([], [], 'r-', label='X')
    imu_plot_y, = axes[3].plot([], [], 'g-', label='Y')
    imu_plot_z, = axes[3].plot([], [], 'b-', label='Z')
    depth_plot = axes[4].bar([0], [0], width=0.5)  # Single bar for depth value

    axes[0].set_title('RGB Frame')
    axes[1].set_title('Flow Feature')
    axes[2].set_title('Additional RGB Feature')
    axes[3].set_title('IMU Data (X, Y, Z)')
    axes[4].set_title('Depth Sensor Data')
    
        # Set a fixed Y-axis range for IMU data
    imu_y_range = (-10, 10)  # Adjust this range based on your IMU data values
    axes[3].set_ylim(imu_y_range)  # Set a fixed range for the IMU Y-axis


    # Turn off axis for cleaner visuals
    for ax in axes:
        ax.axis('off') if ax != axes[3] else ax.set_xticks([])  # Don't turn off axis for IMU

    # IMU axis should show the three axes, so enable labels for this plot
    axes[3].legend(loc='upper right')

    def update(frame):
        # Update RGB data
        rgb_image = rgb_frames[frame].permute(1, 2, 0).cpu().numpy()
        rgb_im.set_data(rgb_image)
        
        # Update flow data
        flow_image = flow_frames[frame].cpu().numpy().reshape(32, 32)
        flow_im.set_data(flow_image)
        flow_im.set_clim(vmin=np.min(flow_image), vmax=np.max(flow_image))  # Set dynamic color range
        
        # Update additional RGB data
        additional_rgb_image = additional_rgb[frame].cpu().numpy().reshape(32, 32)
        add_rgb_im.set_data(additional_rgb_image)
        add_rgb_im.set_clim(vmin=np.min(additional_rgb_image), vmax=np.max(additional_rgb_image))  # Set dynamic color range
        
        # Update IMU data
        imu_values = imu_data[frame].cpu().numpy()  # shape (3,)
        imu_plot_x.set_data([0, 1], [0, imu_values[0]])  # X-axis value
        imu_plot_y.set_data([0, 1], [0, imu_values[1]])  # Y-axis value
        imu_plot_z.set_data([0, 1], [0, imu_values[2]])  # Z-axis value

        # Update depth sensor data
        depth_value = depth_data[frame].cpu().numpy().item()  # Assuming depth_data has shape (num_frames, 1)
        depth_plot[0].set_height(depth_value)
        depth_plot[0].set_y(depth_value)

        return [rgb_im, flow_im, add_rgb_im, imu_plot_x, imu_plot_y, imu_plot_z] + list(depth_plot)

    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=200, blit=True)
    
    # Save as GIF using Pillow backend
    ani.save('./visualizations/animated_video_with_imu_depth.gif', writer='pillow', fps=30)
    
    plt.show()

plot_index = min(rgb_frames.shape[0], flow_frames.shape[0], rgb_feature.shape[0], imu_data.shape[0], depth_data.shape[0])

# Example usage with your batch data
animate_rgb_flow_imu_depth(rgb_frames[0:plot_index], flow_frames[0:plot_index], rgb_feature[0:plot_index], imu_data[0:plot_index], depth_data[0:plot_index])


# Tumbling window frame wise

In [50]:
# import os
# import json
# import math
# import numpy as np
# import torch
# from torch.utils.data import Dataset
# from torchvision import transforms
# from collections import OrderedDict
# import itertools
# from torchvision.io import VideoReader

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
# ])

# def collate_fn(batch, frames_per_clip=30):
#     # Initialize empty lists for all possible modalities
#     padded_audio_clips = []
#     padded_flow_clips = []
#     padded_rgb_clips = []
#     padded_resnet_clips = []
#     padded_resnet_exo_clips = []  # Added for resnet_exo modality
#     padded_smartwatch_clips = []
#     padded_depth_sensor_clips = []
#     keystep_labels = []
#     keystep_ids = []
#     start_frames = []
#     end_frames = []
#     start_ts = []
#     end_ts = []
#     subject_ids = []
#     trial_ids = []

#     # Initialize max lengths for each modality, check if each is available
#     max_flow_len = max([clip['flow'].shape[0] for clip in batch if isinstance(clip.get('flow', None), torch.Tensor)], default=0)
#     max_audio_len = max([clip['audio'].shape[-1] for clip in batch if isinstance(clip.get('audio', None), torch.Tensor)], default=0)
#     max_rgb_len = max([clip['rgb'].shape[0] for clip in batch if isinstance(clip.get('rgb', None), torch.Tensor)], default=0)
#     max_resnet_len = max([clip['resnet'].shape[0] for clip in batch if isinstance(clip.get('resnet', None), torch.Tensor)], default=0)
#     max_resnet_exo_len = max([clip['resnet_exo'].shape[0] for clip in batch if isinstance(clip.get('resnet_exo', None), torch.Tensor)], default=0)  # Added for resnet_exo modality
#     max_smartwatch_len = max([clip['smartwatch'].shape[0] for clip in batch if isinstance(clip.get('smartwatch', None), torch.Tensor)], default=0)
#     max_depth_sensor_len = max([clip['depth_sensor'].shape[0] for clip in batch if isinstance(clip.get('depth_sensor', None), torch.Tensor)], default=0)

#     # print("max_flow_len:",max_flow_len)
#     # print("max_audio_len:",max_audio_len)
#     # print("max_rgb_len:",max_rgb_len)
#     # print("max_resnet_len:",max_resnet_len)
#     # print("max_resnet_exo_len:",max_resnet_exo_len)
#     # print("max_smartwatch_len:",max_smartwatch_len)
#     # print("max_depth_sensor_len:",max_depth_sensor_len)
    
#     pad_length = 0
#     for b in batch:
#         # print("b:",b)
#         # Pad audio if available
#         if 'audio' in b and isinstance(b['audio'], torch.Tensor):
#             audio_clip = b['audio']
#             audio_pad_size = max_audio_len - audio_clip.shape[0]
#             if audio_pad_size > 0:
#                 audio_pad = torch.zeros((audio_pad_size, *audio_clip.shape[1:]))
#                 audio_clip = torch.cat([audio_clip, audio_pad], dim=0)
#             padded_audio_clips.append(audio_clip)


#         # Pad flow data if available
#         if 'flow' in b and isinstance(b['flow'], torch.Tensor):
#             flow_clip = b['flow']
#             flow_pad_size = max_flow_len - flow_clip.shape[0]
#             if flow_pad_size > 0:
#                 flow_pad = torch.zeros((flow_pad_size, *flow_clip.shape[1:]))
#                 flow_clip = torch.cat([flow_clip, flow_pad], dim=0)
#             padded_flow_clips.append(flow_clip)

#         # Pad rgb data if available
#         if 'rgb' in b and isinstance(b['rgb'], torch.Tensor):
#             rgb_clip = b['rgb']
#             rgb_pad_size = max_rgb_len - rgb_clip.shape[0]
#             if rgb_pad_size > 0:
#                 rgb_pad = torch.zeros((rgb_pad_size, *rgb_clip.shape[1:]))
#                 rgb_clip = torch.cat([rgb_clip, rgb_pad], dim=0)
#             padded_rgb_clips.append(rgb_clip)

#         # Pad resnet data if available
#         if 'resnet' in b and isinstance(b['resnet'], torch.Tensor):
#             resnet_clip = b['resnet']
#             resnet_pad_size = max_resnet_len - resnet_clip.shape[0]
#             if resnet_pad_size > 0:
#                 resnet_pad = torch.zeros((resnet_pad_size, *resnet_clip.shape[1:]))
#                 resnet_clip = torch.cat([resnet_clip, resnet_pad], dim=0)
#             padded_resnet_clips.append(resnet_clip)

#         # Pad resnet_exo data if available
#         if 'resnet_exo' in b and isinstance(b['resnet_exo'], torch.Tensor):
#             resnet_exo_clip = b['resnet_exo']
#             resnet_exo_pad_size = max_resnet_exo_len - resnet_exo_clip.shape[0]
#             if resnet_exo_pad_size > 0:
#                 resnet_exo_pad = torch.zeros((resnet_exo_pad_size, *resnet_exo_clip.shape[1:]))
#                 resnet_exo_clip = torch.cat([resnet_exo_clip, resnet_exo_pad], dim=0)
#             padded_resnet_exo_clips.append(resnet_exo_clip)

#         # Pad smartwatch data if available
#         if 'smartwatch' in b and isinstance(b['smartwatch'], torch.Tensor):
#             smartwatch_clip = b['smartwatch']
#             if max_smartwatch_len > 0:
#                 smartwatch_pad_size = max_smartwatch_len - smartwatch_clip.shape[0]
#                 if smartwatch_pad_size > 0:
#                     smartwatch_pad = torch.zeros((smartwatch_clip.shape[0], smartwatch_pad_size))
#                     smartwatch_clip = torch.cat([smartwatch_clip, smartwatch_pad], dim=0)
#             padded_smartwatch_clips.append(smartwatch_clip)

#         # Pad depth_sensor data if available
#         if 'depth_sensor' in b and isinstance(b['depth_sensor'], torch.Tensor):
#             depth_sensor_clip = b['depth_sensor']
#             if max_depth_sensor_len > 0:
#                 depth_sensor_pad_size = max_depth_sensor_len - depth_sensor_clip.shape[0]
#                 if depth_sensor_pad_size > 0:
#                     depth_sensor_pad = torch.zeros((depth_sensor_clip.shape[0], depth_sensor_pad_size))
#                     depth_sensor_clip = torch.cat([depth_sensor_clip, depth_sensor_pad], dim=0)
#             padded_depth_sensor_clips.append(depth_sensor_clip)


#         pad_length = frames_per_clip - len(b['keystep_id']) 
        
#         if pad_length > 0:
#             # repeat last element of keystep_id and keystep_label
#             b['keystep_id'] = b['keystep_id'] + [b['keystep_id'][-1]]*pad_length
#             b['start_frame'] = b['start_frame'] + [b['start_frame'][-1]]*pad_length
#             b['end_frame'] = b['end_frame'] + [b['end_frame'][-1]]*pad_length
#             b['start_t'] = b['start_t'] + [b['start_t'][-1]]*pad_length
#             b['end_t'] = b['end_t'] + [b['end_t'][-1]]*pad_length
        
#         # Collect other fields
#         keystep_labels.append(b['keystep_label'])
#         keystep_ids.append(b['keystep_id'])
#         start_frames.append(b['start_frame'])
#         end_frames.append(b['end_frame'])
#         start_ts.append(b['start_t'])
#         end_ts.append(b['end_t'])
#         subject_ids.append(b['subject_id'])
#         trial_ids.append(b['trial_id'])

#     output = {
#         'keystep_label': keystep_labels,
#         'keystep_id': torch.tensor(keystep_ids),
#         'start_frame': torch.tensor(start_frames),
#         'end_frame': torch.tensor(end_frames),
#         'start_t': torch.tensor(start_ts),
#         'end_t': torch.tensor(end_ts),
#         'subject_id': subject_ids,
#         'trial_id': trial_ids
#     }

#     # Only include modality data if it exists
#     output['audio'] = torch.stack(padded_audio_clips) if padded_audio_clips else torch.zeros(0)
#     output['flow'] = torch.stack(padded_flow_clips) if padded_flow_clips else torch.zeros(0)
#     output['rgb'] = torch.stack(padded_rgb_clips) if padded_rgb_clips else torch.zeros(0)
#     output['resnet'] = torch.stack(padded_resnet_clips) if padded_resnet_clips else torch.zeros(0)
#     output['resnet_exo'] = torch.stack(padded_resnet_exo_clips) if padded_resnet_exo_clips else torch.zeros(0)  # Added for resnet_exo
#     output['smartwatch'] = torch.stack(padded_smartwatch_clips) if padded_smartwatch_clips else torch.zeros(0)
#     output['depth_sensor'] = torch.stack(padded_depth_sensor_clips) if padded_depth_sensor_clips else torch.zeros(0)

#     return output


# class WindowEgoExoEMSDataset(Dataset):
#     def __init__(self, annotation_file, data_base_path, fps, 
#                 frames_per_clip=30, transform=None,
#                 data_types=['resnet']):
        
#         self.annotation_file = annotation_file
#         self.data_base_path = data_base_path
#         self.fps = fps
#         self.frames_per_clip = frames_per_clip  # Store frames_per_clip
#         self.transform = transform
#         self.data = []
#         self.clip_indices = []  # This will store (item_idx, clip_idx) tuples
#         self.data_types = data_types
        
#         self.data_dict = None
#         self._load_annotations()
#         self._split_windows()

#     def _load_annotations(self):
#         with open(self.annotation_file, 'r') as f:
#             annotations = json.load(f)
        
#         subject_dict = {}
#         for subject in annotations['subjects']:
#             trial_dict ={}
#             for trial in subject['trials']:
#                 avail_streams = trial['streams']
                
#                 # Initialize paths to None by default
#                 audio_path = None
#                 flow_path = None
#                 rgb_path = None
#                 resnet_path = None
#                 resnet_exo_path = None  # Added for resnet_exo modality
#                 smartwatch_path = None
#                 depth_sensor_path = None

#                 # Check for each data type and retrieve the corresponding file path
#                 if 'audio' in self.data_types:
#                     audio_path = avail_streams.get('egocam_rgb_audio', {}).get('file_path', None)
#                 if 'flow' in self.data_types:
#                     flow_path = avail_streams.get('i3d_flow', {}).get('file_path', None)
#                 if 'rgb' in self.data_types:
#                     rgb_path = avail_streams.get('i3d_rgb', {}).get('file_path', None)
#                 if 'resnet' in self.data_types:
#                     resnet_path = avail_streams.get('resnet50', {}).get('file_path', None)
#                 if 'resnet_exo' in self.data_types:
#                     resnet_exo_path = avail_streams.get('resnet50-exo', {}).get('file_path', None)  # Adjust key as needed
#                 if 'smartwatch' in self.data_types:
#                     smartwatch_path = avail_streams.get('smartwatch_imu', {}).get('file_path', None)
#                 if 'depth_sensor' in self.data_types:
#                     depth_sensor_path = avail_streams.get('vl6180_ToF_depth', {}).get('file_path', None)

#                 # Skip the trial if any required data type is not available
#                 if ('flow' in self.data_types and not flow_path) or \
#                 ('audio' in self.data_types and not audio_path) or \
#                 ('rgb' in self.data_types and not rgb_path) or \
#                 ('resnet' in self.data_types and not resnet_path) or \
#                 ('resnet_exo' in self.data_types and not resnet_exo_path) or \
#                 ('smartwatch' in self.data_types and not smartwatch_path) or \
#                 ('depth_sensor' in self.data_types and not depth_sensor_path):
#                     print(f"[Warning] Skipping trial {trial['trial_id']} for subject {subject['subject_id']} due to missing data")
#                     continue
                
#                 if audio_path or flow_path or rgb_path or resnet_path or resnet_exo_path or smartwatch_path or depth_sensor_path:
#                     keysteps = trial['keysteps']
#                     keysteps_dict = []
#                     for step in keysteps:
#                         start_frame = math.floor(step['start_t'] * self.fps)
#                         end_frame = math.floor(step['end_t'] * self.fps)
#                         label = step['label']
#                         keystep_id = step['class_id']

#                         data_dict = {}
#                         if 'audio' in self.data_types:
#                             data_dict['audio_path'] = os.path.join(self.data_base_path, audio_path)
#                         if 'flow' in self.data_types:
#                             data_dict['flow_path'] = os.path.join(self.data_base_path, flow_path)
#                         if 'rgb' in self.data_types:
#                             data_dict['rgb_path'] = os.path.join(self.data_base_path, rgb_path)
#                         if 'resnet' in self.data_types:
#                             data_dict['resnet_path'] = os.path.join(self.data_base_path, resnet_path)
#                         if 'resnet_exo' in self.data_types:
#                             data_dict['resnet_exo_path'] = os.path.join(self.data_base_path, resnet_exo_path)
#                         if 'smartwatch' in self.data_types:
#                             data_dict['smartwatch_path'] = os.path.join(self.data_base_path, smartwatch_path)
#                         if 'depth_sensor' in self.data_types:
#                             data_dict['depth_sensor_path'] = os.path.join(self.data_base_path, depth_sensor_path)
#                         data_dict['start_frame'] = start_frame
#                         data_dict['end_frame'] = end_frame
#                         data_dict['start_t'] = step['start_t']
#                         data_dict['end_t'] = step['end_t']
#                         data_dict['keystep_label'] = label
#                         data_dict['keystep_id'] = keystep_id
#                         data_dict['subject'] = subject['subject_id']
#                         data_dict['trial'] = trial['trial_id']

#                         keysteps_dict.append(data_dict)
#                         self.data.append(data_dict)

#                         trial_dict[trial['trial_id']] = keysteps_dict
#                 subject_dict[subject['subject_id']] = trial_dict

#         self.data_dict = subject_dict

#     def _split_windows(self):
#         print("Splitting data to windows")
#         windowed_clips = []
#         current_window = []  # Initialize an empty current window
#         accumulated_frames = 0  # Track how many frames have been accumulated in the current window

#         current_subject = None
#         current_trial = None


#         for i, item in enumerate(self.data):
#             start_frame = item['start_frame']
#             end_frame = item['end_frame']
#             keystep_id = item['keystep_id']
#             keystep_label = item['keystep_label']
#             subject_id = item['subject']
#             trial_id = item['trial']
#             num_frames_total = end_frame - start_frame

                
#         # If the subject or trial changes, store the current window and reset it
#             if subject_id != current_subject or trial_id != current_trial:
#                 if len(current_window) > 0:
#                     windowed_clips.append(current_window)
#                 current_window = []  # Reset the current window
#                 accumulated_frames = 0  # Reset the frame counter
#                 current_subject = subject_id  # Update current subject
#                 current_trial = trial_id  # Update current trial

#             # Loop through frames from the start to the end of the current keystep
#             for j in range(start_frame, end_frame):
#                 frame_data = {}
#                 frame_data['frame'] = j

#                 # Copy the relevant data for the current frame
#                 for key, value in item.items():
#                     if isinstance(value, (int, float, str)):
#                         frame_data[key] = value
#                     elif isinstance(value, (list, np.ndarray)):
#                         if len(value) == num_frames_total:
#                             frame_data[key] = value[j - start_frame]
#                     else:
#                         frame_data[key] = value

#                 # Append the current frame data to the window
#                 current_window.append(frame_data)
#                 accumulated_frames += 1


#                 # Once we reach the window size, store the window and reset
#                 if accumulated_frames == self.frames_per_clip:
#                     windowed_clips.append(current_window)
#                     current_window = []  # Reset the current window
#                     accumulated_frames = 0  # Reset the frame counter

#         # # dump the data to a file
#         # with open('data.json', 'w') as f:
#         #     json.dump(windowed_clips, f)
            
#         self.data = windowed_clips
#         print(f"Total windowed clips: {len(windowed_clips)}")

            

#     def __len__(self):
#         # The length should now be based on the number of clips, not items
#         # total_clips = len(self.data) // self.frames_per_clip if self.frames_per_clip else len(self.data)
#         # dump the data to a file
#         # with open('data.json', 'w') as f:
#         #     json.dump(self.data, f)
#         total_clips = len(self.data)
#         return total_clips

#     def __getitem__(self, idx):
#         window = self.data[idx]
#         # print("window", window)

#         first_frame_of_clip = window[0]['frame']
#         last_frame_of_clip = window[-1]['frame']+1
        
#         # print(f"getting clip {idx} with frame {first_frame_of_clip} to {last_frame_of_clip} of length ({last_frame_of_clip - first_frame_of_clip})")

#         # Initialize dictionaries to hold accumulated frames for each modality
#         batch_audio = []
#         batch_flow = []
#         batch_rgb = []
#         batch_resnet = []
#         batch_resnet_exo = []
#         batch_sw_acc = []
#         batch_depth_sensor = []

#         # Initialize lists for metadata
#         keystep_labels = []
#         keystep_ids = []
#         start_frames = []
#         end_frames = []
#         start_ts = []
#         end_ts = []
#         subject_ids = []
#         trial_ids = []


#         # Initialize variables
#         flow = torch.zeros(0)
#         rgb = torch.zeros(0)
#         resnet = torch.zeros(0)
#         resnet_exo = torch.zeros(0)  # Added for resnet_exo modality
#         sw_acc = torch.zeros(0)
#         depth_sensor_readings = torch.zeros(0)

#         # Load flow if available
#         if 'audio' in self.data_types:
#             audio_path = window[0]['audio_path']
#             clip_start_t = first_frame_of_clip / self.fps
#             clip_end_t = last_frame_of_clip / self.fps

#             # print(f"loading audio from {clip_start_t} to {clip_end_t} from file {audio_path}")

#             audio_reader = VideoReader(audio_path, "audio")
#             audio_clips = []
#             for audio_frame in itertools.takewhile(lambda x: x['pts'] <= clip_end_t, audio_reader.seek(clip_start_t)):
#                 audio_clips.append(audio_frame['data'])
#             audio_clips = torch.cat(audio_clips, dim=0) if audio_clips else torch.zeros(1, 0)

#             # pad the flow tensor to the i
#             batch_audio.append(audio_clips)


#         if 'flow' in self.data_types:
#             flow_path = window[0]['flow_path']
#             flow_npy = np.load(flow_path)
#             flow_length = len(flow_npy)
#             flow = torch.from_numpy(np.load(flow_path))[first_frame_of_clip:last_frame_of_clip]
#                 # pad the flow tensor to the i
#             batch_flow.append(flow)

#         # Load rgb if available
#         if 'rgb' in self.data_types:
#             rgb_path =  window[0]['rgb_path']
#             rgb_npy = np.load(rgb_path)
#             rgb_length = len(rgb_npy)
#             rgb = torch.from_numpy(np.load(rgb_path))[first_frame_of_clip:last_frame_of_clip]
#                 # pad the rgb tensor to the i
#             batch_rgb.append(rgb)

#         # Load resnet if available
#         if 'resnet' in self.data_types:
#             resnet_path =  window[0]['resnet_path']
#             resnet_npy = np.load(resnet_path)
#             resnet_length = len(resnet_npy)
#             resnet = torch.from_numpy(np.load(resnet_path))[first_frame_of_clip:last_frame_of_clip]
#             batch_resnet.append(resnet)

#         # Load resnet_exo if available
#         if 'resnet_exo' in self.data_types:
#             resnet_exo_path =  window[0]['resnet_exo_path']
#             resnet_exo = torch.from_numpy(np.load(resnet_exo_path))[first_frame_of_clip:last_frame_of_clip]
#             batch_resnet_exo.append(resnet_exo)

#         # Load smartwatch data if available
#         if 'smartwatch' in self.data_types:
#             smartwatch_path =  window[0]['smartwatch_path']
#             with open(smartwatch_path, 'r') as f:
#                 lines = f.readlines()[1:]
#             sw_acc = [line.strip() for line in lines][first_frame_of_clip:last_frame_of_clip]
#             acc_x = [float(l.split(',')[0]) for l in sw_acc]
#             acc_y = [float(l.split(',')[1]) for l in sw_acc]
#             acc_z = [float(l.split(',')[2]) for l in sw_acc]
#             sw_acc = torch.from_numpy(np.array([acc_x, acc_y, acc_z])).float()
#             sw_acc = sw_acc.permute(1, 0)  # (frames, channels)
#             batch_sw_acc.append(sw_acc)

#         # Load depth_sensor data if available
#         if 'depth_sensor' in self.data_types:
#             depth_sensor_path =  window[0]['depth_sensor_path']
#             with open(depth_sensor_path, 'r') as f:
#                 lines = f.readlines()[1:]
#             depth_sensor_readings = [line.strip() for line in lines][first_frame_of_clip:last_frame_of_clip]
#             depth_reading = [float(l.split(',')[0]) for l in depth_sensor_readings]
#             depth_sensor_readings = torch.from_numpy(np.array([depth_reading])).float()
#             depth_sensor_readings = depth_sensor_readings.permute(1, 0)
#             batch_depth_sensor.append(depth_sensor_readings)

#         for frame in window:
#             # Accumulate metadata for each frame
#             keystep_labels.append( frame['keystep_label'])
#             keystep_ids.append( frame['keystep_id'])
#             start_frames.append( frame['start_frame'])
#             end_frames.append( frame['end_frame'])
#             start_ts.append( frame['start_t'])
#             end_ts.append( frame['end_t'])
#             subject_ids.append( frame['subject'])
#             trial_ids.append( frame['trial'])

#         # print("batch_resnet:",batch_resnet[0].shape)
#         # Stack frames to form a batch of frames_per_clip
#         output = {
#             'audio': (batch_audio[0]) if batch_audio else torch.zeros(1,0),
#             'flow': (batch_flow[0]) if batch_flow else torch.zeros(0),
#             'rgb': (batch_rgb[0]) if batch_rgb else torch.zeros(0),
#             'resnet': (batch_resnet[0]) if batch_resnet else torch.zeros(0),
#             'resnet_exo': (batch_resnet_exo[0]) if batch_resnet_exo else torch.zeros(0),
#             'smartwatch': (batch_sw_acc[0]) if batch_sw_acc else torch.zeros(0),
#             'depth_sensor': (batch_depth_sensor[0]) if batch_depth_sensor else torch.zeros(0),

#             # Metadata (individual per frame)
#             'keystep_label': keystep_labels,
#             'keystep_id': keystep_ids,
#             'start_frame': start_frames,
#             'end_frame': end_frames,
#             'start_t': start_ts,
#             'end_t': end_ts,
#             'subject_id': subject_ids,
#             'trial_id': trial_ids
#         }

#         return output



In [1]:
from functools import partial

from EgoExoEMS.EgoExoEMS import *

annotation_file = "../../Annotations/main_annotation_segmentation.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)

# train_annotation_file = "../../Annotations/splits/keysteps/train_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)
# val_annotation_file = "../../Annotations/splits/keysteps/val_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)
# test_annotation_file = "../../Annotations/splits/keysteps/test_split.json"  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)

train_dataset = WindowEgoExoEMSDataset(annotation_file=annotation_file,
                                data_base_path='',
                                fps=29.97, frames_per_clip=120, transform=transform, data_types=[ 'flow', 'rgb'])


print("Size of dataset:",len(train_dataset))

# Use a partial function or lambda to pass the frames_per_clip argument
collate_fn_with_args = partial(window_collate_fn, frames_per_clip=120)


data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn_with_args)

print("Size of data loader:",len(data_loader))

# Access a sample
# batch = next(iter(data_loader))
# print(batch['resnet'].shape, batch['resnet_exo'].shape, batch['flow'].shape, batch['rgb'].shape, batch['smartwatch'].shape, batch['depth_sensor'].shape , batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'],batch['start_t'], batch['end_t'],  batch['subject_id'], batch['trial_id'])

for idx,batch in enumerate(data_loader):
    print("batch", batch['resnet'].shape, batch['smartwatch'].shape)
    # print(batch['resnet'].shape, batch['audio'].shape, batch['resnet_exo'].shape, batch['flow'].shape, batch['rgb'].shape, batch['smartwatch'].shape, batch['depth_sensor'].shape , batch['keystep_label'], batch['keystep_id'], batch['start_frame'], batch['end_frame'],batch['start_t'], batch['end_t'],  batch['subject_id'], batch['trial_id'])
    # print(batch['resnet'].shape)
    # break
    

Splitting data to windows
Total windowed clips: 2498
Size of dataset: 2498
Size of data loader: 500
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120]) torch.Size([5, 0])
batch torch.Size([5, 120])

KeyboardInterrupt: 